# SPEG Quality Indicator Calculations

## Turner Syndrome Quality Indicators


The source of the data is the Tableau Master Report from BOXI and the Endocrine Report which cannot be retrieved directly from BOXI but must be specifically requested.

There are 5 Turner Syndrome QIs to be calculated.
1. Growth Hormone Treatment - Yes/No values
2. 6 Monthly Height Recording - Calculated 6 months back from current date
3. Annual IGF-1 Measurement - Calculated 12 months back from current date
4. Oestrogen Replacement Therapy (ORT) - Yes/No values
5. ORT Started at Age 10-12 - Calculated using patient's age at time of start date

##### First, the data is retrieved and assigned to dataframes

In [9]:
# import necessary libraries for analysis
import pandas as pd
import numpy as np
from datetime import datetime
import sys
import os


In [10]:
# define the pathway where source data is retrieved
file_input_path = 'data'

# directory to store excel files
output_directory = 'health_board_files/'

In [11]:
# assign the core data to a dataframe
raw_core_df = pd.read_excel(file_input_path + '/2024-07-30 ENDOCRINE-Report.xlsx', sheet_name='CoreDataset')


# assign the demographic data to a dataframe
raw_demographic_df = pd.read_excel(file_input_path + '/2024-07-30 SPEG Tableau Master Report v2.xlsx', sheet_name='Demographics')


# assign the demographic data to a dataframe
raw_condition_df = pd.read_excel(file_input_path + '/2024-07-30 SPEG Tableau Master Report v2.xlsx', sheet_name='Condition')


### Clean core dataset

In [12]:
# copy raw data to new dataframes which will be used for cleaning
core_df = raw_core_df
demographic_df = raw_demographic_df
condition_df = raw_condition_df


In [13]:
# we only want to keep patients where their status is Current or Blank
demographic_df = demographic_df[(demographic_df['Patient Status'] == 'CURRENT') | pd.isna(demographic_df['Patient Status'])]


In [14]:
# we don't need every column contained in the data for calculating QIs. Following defines which ones to drop
columns_to_drop = ['Surname', 'Forename',
                   'NGHTR', 'NGHTRO',
                   'DOTTE', 'DOLTTE_MR',
                   'DOBP', 'DOAA',
                   'DOCEFS', 'DOVDS', 
                   'DOCS', 'DOLFT', 
                   'DORUSSAD', 'DODBS',
                   'DOATFTS', 'DOAHbA1cS', 'DOOA']


# assign the column dropping list
core_df = core_df.drop(columns=columns_to_drop)


In [15]:
# rename columns appropriately from their key to actual name
column_renaming = {'CHINumber': 'CHI', 
                   'HealthBoardOfResidence': 'Health_Board', 
                   'TreatmentCentre': 'Treatment_Centre', 
                   'DOR': 'Date_Of_Record', 
                   'GHT': 'Growth_Hormone_Treatment', 
                   'DHR': 'Date_Height_Recorded', 
                   'DIGF1D': 'Date_IGF1_Done', 
                   'ORT': 'Oestrogen_Replacement_Therapy', 
                   'ORSD': 'Oestrogen_Replacement_StartDate'}


 # assign the column renaming list
core_df = core_df.rename(columns=column_renaming)                  


In [16]:
# after discovering some erroneous values in the growth hormone column which should only contain 0 or 1, these can be removed
core_df['Growth_Hormone_Treatment'] = core_df['Growth_Hormone_Treatment'].replace({val: np.nan for val in core_df['Growth_Hormone_Treatment'] if val not in [0, 1]})


In [17]:
# the test patient with a CHI of 1111111111 is contained in this data and must be removed
core_df = core_df[core_df['CHI'] != 1111111111]


In [18]:
# where CDS_Deleted or PC_Deleted is 1, these rows can be removed
core_df = core_df[(core_df['CDS_Deleted'] != 1) & (core_df['PC_Deleted'] != 1)]


In [19]:
# somewhere in the process of the endocrine report being produced, the leading zeros of CHI numbers have been ommitted
# the following adds these back in
core_df.loc[:, 'CHI'] = core_df['CHI'].astype(str).apply(lambda x: x.zfill(10) if len(x) == 9 else x)


In [20]:
# we don't need every column contained in the data for calculating QIs. Following defines which ones to drop
columns_to_drop = ['Treatment_Centre',
                   'PC_Deleted',
                   'CDS_Deleted',
                   'Health_Board']


# assign the column dropping list
core_df = core_df.drop(columns=columns_to_drop)


### Clean then combine demographic_df and condition_df

In [21]:
# add leading zero to chi
condition_df.loc[:, 'Chi Number'] = condition_df['Chi Number'].astype(str).apply(lambda x: x.zfill(10) if len(x) == 9 else x)


# add leading zero to chi
demographic_df.loc[:, 'Chi Number'] = demographic_df['Chi Number'].astype(str).apply(lambda x: x.zfill(10) if len(x) == 9 else x)

In [22]:
# where Condition Deleted is 1, these rows can be removed
condition_df = condition_df[(condition_df['Condition Deleted'] != 1)]


In [23]:
# Combine first and second name into a single column
demographic_df.loc[:, 'Name'] = demographic_df['First Forename'] + ' ' + demographic_df['Surname']


# extract last name initial 
demographic_df = demographic_df.assign(SurnameInitial=demographic_df['Surname'].apply(lambda x: x[0]))


# drop the initial columns
demographic_df = demographic_df.drop(columns=['First Forename', 'Surname'])

C:\Users\scotth07\AppData\Local\Temp\ipykernel_18544\2587080016.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demographic_df.loc[:, 'Name'] = demographic_df['First Forename'] + ' ' + demographic_df['Surname']


In [24]:
# # define list of columns to drop from condition_df
# drop_demographic_columns = ['First Forename',
#                             'Surname',
#                             'Patient Created Date']

# # drop the columns
# demographic_df = demographic_df.drop(columns=drop_demographic_columns)

In [25]:
# define list of columns to drop from condition_df
drop_condition_columns = ['Condition Outcome',
                          'Condition Outcome Date',
                          'Condition Deleted', 
                          'Date Condition Diagnosed']

# drop the columns
condition_df = condition_df.drop(columns=drop_condition_columns)

In [26]:
# merge the demographics and condition dataframes
ts_demographics_df = pd.merge(demographic_df, condition_df, on='Chi Number', how='left')

In [27]:
# filter the merged dataframe to only include turner syndrome patients
ts_demographics_df = ts_demographics_df[(ts_demographics_df['Condition.Condition'] == "TURNER'S SYNDROME") | (ts_demographics_df['Sub Condition'] == "SYNDROMIC - TURNER'S")]


# reset the index
ts_demographics_df = ts_demographics_df.reset_index(drop=True)

In [28]:
# drop rows where there are duplicate chi numbers
ts_demographics_df = ts_demographics_df.drop_duplicates(subset='Chi Number')


# reset index
ts_demographics_df = ts_demographics_df.reset_index(drop=True)

In [29]:
# rename columns appropriately
column_renaming = {'Chi Number': 'CHI',
                   'Date Of Birth': 'Date_of_Birth',
                   'Health Board Of Residence Description': 'Health_Board_Of_Residence',
                   'Patient Status': 'Patient_Status', 
                   'Patient Treatment Centre': 'Treatment_Centre',
                   'Patient Treatment Centre Health Board': 'Treatment_Centre_Health_Board', 
                   'Condition.Condition': 'Condition',
                   'Sub Condition': 'Sub_Condition'}


# assign the column renaming list
ts_demographics_df = ts_demographics_df.rename(columns=column_renaming)                  


### Calculate QIS

#### 1. Growth Hormone Treatment

Prioritises most recent date of record

In [30]:
# merge the two dataframes before calculating QIs
merged_df = pd.merge(ts_demographics_df, core_df, on='CHI', how='left')


# most recent date of record if prioritised, the dataframe is sorted accordingly
merged_df = merged_df.sort_values(by='Date_Of_Record', ascending=False)


# drop duplicate rows so we can retrieve a single value for each patient
merged_df = merged_df.drop_duplicates(subset='CHI', keep='first')

In [31]:
# retrieve value of Growth_Hormone_Treatment for each patient using CHI value as key
ts_demographics_df['Growth_Hormone_Treatment'] = ts_demographics_df['CHI'].map(merged_df.set_index('CHI')['Growth_Hormone_Treatment'])


# replace values of 1 with True and 0 with False
ts_demographics_df['Growth_Hormone_Treatment'] = ts_demographics_df['Growth_Hormone_Treatment'].replace({1:True, 0:False})

#### 2. 6 Monthly Height Recording

Prioritises most recent date height recorded

In [32]:
# assign todays date to variable
today = pd.to_datetime('today')


# assign date of 6 months before today to variable
six_months_ago = today - pd.DateOffset(months=6)


In [33]:
def Six_Monthly_Height_Recording(row):
    if row['Date_Height_Recorded'] >= six_months_ago:
        val = True
    elif row['Date_Height_Recorded'] < six_months_ago:
        val = False
    else:
        val = np.nan
    return val

In [34]:
# merge the two dataframes before calculating QIs
merged_df = pd.merge(ts_demographics_df, core_df, on='CHI', how='left')


# most recent date of record if prioritised, the dataframe is sorted accordingly
merged_df = merged_df.sort_values(by='Date_Of_Record', ascending=False)


# drop duplicate rows so we can retrieve a single value for each patient
merged_df = merged_df.drop_duplicates(subset='CHI', keep='first')


# apply function to return true or false based on date height recorded
merged_df['6_Monthly_Height_Recording'] = merged_df.apply(Six_Monthly_Height_Recording, axis=1)


In [35]:
# merge the dataframes now that the 6 monthly height recording QI has been calculated
ts_demographics_df = pd.merge(ts_demographics_df, merged_df[['CHI', '6_Monthly_Height_Recording']], on='CHI', how='left')


#### 3. Annual IGF-1 Measurement

Prioritises most recent date IGF-1 measured

In [36]:
# assign date 12 months before today to variable
one_year_ago = today - pd.DateOffset(months=12)


In [37]:
def Annual_IFG1_Measurement(row):
    if row['Date_IGF1_Done'] >= one_year_ago:
        val = True
    elif row['Date_IGF1_Done'] < one_year_ago:
        val = False
    else:
        val = np.nan
    return val

In [38]:
# merge the two dataframes before calculating QIs
merged_df = pd.merge(ts_demographics_df, core_df, on='CHI', how='left')


# most recent date of record if prioritised, the dataframe is sorted accordingly
merged_df = merged_df.sort_values(by='Date_Of_Record', ascending=False)


# drop duplicate rows so we can retrieve a single value for each patient
merged_df = merged_df.drop_duplicates(subset='CHI', keep='first')


# apply function to return true or false based on date height recorded
merged_df['Annual_IGF1_Measurement'] = merged_df.apply(Annual_IFG1_Measurement, axis=1)


In [39]:
# merge the dataframes now that the 6 monthly height recording QI has been calculated
ts_demographics_df = pd.merge(ts_demographics_df, merged_df[['CHI', 'Annual_IGF1_Measurement']], on='CHI', how='left')


#### 4. Oestrogen Replacement Therapy (ORT)

Prioritises most recent date of record

In [40]:
# merge the two dataframes again to begin fresh with calculations
merged_df = pd.merge(ts_demographics_df, core_df, on='CHI', how='left')


# values of 1 should be prioritised so the dataframe is sorted accordingly
merged_df = merged_df.sort_values(by='Date_Of_Record', ascending=False)


# drop duplicate rows so we can retrieve a single value for each patient
merged_df = merged_df.drop_duplicates(subset='CHI', keep='first')


In [41]:
# retrieve value of Growth_Hormone_Treatment for each patient using CHI value as key
ts_demographics_df['Oestrogen_Replacement_Therapy'] = ts_demographics_df['CHI'].map(merged_df.set_index('CHI')['Oestrogen_Replacement_Therapy'])


# replace values of 1 with True and 0 with False
ts_demographics_df['Oestrogen_Replacement_Therapy'] = ts_demographics_df['Oestrogen_Replacement_Therapy'].replace({1:True, 0:False})

#### 5. ORT Started at Age 10-12

Prioritises most recent date ORT started

In [42]:
# merge the two dataframes again to begin fresh with calculations
merged_df = pd.merge(ts_demographics_df, core_df, on='CHI', how='left')


# dataframe is sorted accordingly
merged_df = merged_df.sort_values(by='Date_Of_Record', ascending=False)


# drop duplicate rows so we can retrieve a single value for each patient
merged_df = merged_df.drop_duplicates(subset='CHI', keep='first')


# calculate the age of the patient when ORT was started
merged_df['ORT_Start_Age'] =  np.floor((merged_df['Oestrogen_Replacement_StartDate'] - merged_df['Date_of_Birth']).dt.days / 365.25)


# return true if patient was between ages 10-12 and false if not
merged_df['ORT_Started_Age_10_12'] = (merged_df['ORT_Start_Age'] <= 12) & (merged_df['ORT_Start_Age'] >= 10)

In [43]:
# where there is no date for ORT, assign value of None
merged_df['ORT_Started_Age_10_12'] = np.where(merged_df['Oestrogen_Replacement_StartDate'].isnull(), None, merged_df['ORT_Started_Age_10_12'])


# replcae values to contain True, False or NaN
merged_df['ORT_Started_Age_10_12'] = merged_df['ORT_Started_Age_10_12'].replace({1:True, 0:False, None:np.nan})

In [44]:
# merge the dataframes
ts_demographics_df = pd.merge(ts_demographics_df, merged_df[['CHI', 'ORT_Started_Age_10_12']], on='CHI', how='left')


#### Finally, I want to check which patients have no core dataset added

In [45]:
ts_demographics_df['Has_Core_Data_Set'] = ts_demographics_df['CHI'].isin(core_df['CHI'])


## Adrenal Insufficiency Quality Indicators


The source of the data is the Tableau Master Report and the Dashboard Data report, both from BOXI

There are 6 Adrenal Insufficiencies QIs to be calculated.
1. Initial Emergency Plan - Yes if there is a date populated, No if not
2. Parental Education On IM Injections - Yes if there is a date populated, No if not
3. Relevant Acute Services Notification - Yes if there is a date populated, No if not
4. School Education - Yes if there is a date populated, No if not
5. Scottish Ambulance Service Notification - Yes if there is a date populated, No if not
6. Annual Update of Emergency Plan - Calculated 12 months back from current date

#### First, the data is retrieved and assigned to dataframes

In [46]:
# define the pathway where source data is retrieved
file_input_path = 'data'


In [47]:
# assign the QI encounter data to a dataframe
raw_qi_df = pd.read_excel(file_input_path + '/2024-07-30 SPEG-Dashboard-Data.xlsx', sheet_name='Encounters Outcomes')


# assign the demographic data to a dataframe
raw_demographic_df = pd.read_excel(file_input_path + '/2024-07-30 SPEG Tableau Master Report v2.xlsx', sheet_name='Demographics')


# assign the condition data to a dataframe
raw_condition_df = pd.read_excel(file_input_path + '/2024-07-30 SPEG Tableau Master Report v2.xlsx', sheet_name='Condition')


C:\Users\scotth07\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [48]:
# copy raw data to new dataframes which will be used for cleaning
qi_df = raw_qi_df
demographic_df = raw_demographic_df
condition_df = raw_condition_df

### Clean dataframes

In [49]:
# column data types must be changed from object to datetime
change_columns = ['ANNUAL UPDATE OF EMERGENCY PLAN',
                  'INITIAL EMERGENCY PLAN',
                  'PARENTAL EDUCATION ON IM INJECTIONS',
                  'RELEVANT ACUTE SERVICES NOTIFICATION',
                  'SCHOOL EDUCATION',
                  'SCOT AMBULANCE SERVICE NOTIFICATION']


# assign the column type change
qi_df[change_columns] = qi_df[change_columns].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y', errors='coerce'))


In [50]:
# replace all values of X with NaN
qi_df = qi_df.replace('X', np.nan)


# change naming of CHI column to match other dataframes 
column_renaming = {'Chi Number': 'CHI'}

qi_df = qi_df.rename(columns=column_renaming)

In [51]:
# we only want to keep patients where their status is Current or Blank
demographic_df = demographic_df[(demographic_df['Patient Status'] == 'CURRENT') | pd.isna(demographic_df['Patient Status'])]


In [52]:
qi_df

,CHI,ANNUAL UPDATE OF EMERGENCY PLAN,ARGININE,CLINIC REVIEW,COMPLETED,DISCHARGE,GP LETTER SENT,INITIAL EMERGENCY PLAN,NOT COMPLETED,PARENTAL EDUCATION ON IM INJECTIONS,...,REFER - TRANSITION CLINIC,RELEVANT ACUTE SERVICES NOTIFICATION,RETURN VISIT - DOCTOR REVIEW,RETURN VISIT - NURSE REVIEW,SCHOOL EDUCATION,SCOT AMBULANCE SERVICE NOTIFICATION,SEVERE GDH ONGOING: GH DISCONTINUED,SHARED CARE FORM SENT TO GP,STERIOD REPLACEMENT MANAGEMENT PLAN,Unnamed: 20
0,101005083,NaT,01/08/2012,NaN,NaN,NaN,30/07/2012,NaT,28/01/2015,NaT,...,NaN,NaT,NaN,NaN,NaT,NaT,10/02/2015,30/07/2012,NaN,02/08/2012
1,101066643,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,...,NaN,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,20/02/2023
2,101070438,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,...,NaN,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,26/01/2021
3,101078587,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,...,NaN,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,14/07/2020
4,101090811,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,...,NaN,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,14/03/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3076,3112058038,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,...,NaN,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,24/06/2022
3077,3112060334,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,...,NaN,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,09/09/2022
3078,3112088212,2024-04-03,NaN,NaN,NaN,NaN,NaN,2024-04-03,NaN,2024-04-03,...,NaN,2024-04-03,NaN,NaN,NaT,2024-04-03,NaN,NaN,NaN,14/11/2023
3079,3112135644,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,...,NaN,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaN,29/11/2022


In [53]:
# we don't need every column contained in the data for calculating QIs. Following defines which ones to drop
columns_to_drop = ['ARGININE',
                   'DISCHARGE',
                   'CLINIC REVIEW',
                   'COMPLETED',
                   'GP LETTER SENT',
                   'NOT COMPLETED',
                   'REFER - OTHER SPECIALTY',
                   'REFER - TRANSITION CLINIC',
                   'RETURN VISIT - DOCTOR REVIEW',
                   'SEVERE GDH ONGOING: GH DISCONTINUED',
                   'SHARED CARE FORM SENT TO GP',
                   'Unnamed: 20',
                   'STERIOD REPLACEMENT MANAGEMENT PLAN']


# assign the column dropping list
qi_df = qi_df.drop(columns=columns_to_drop)

In [54]:
# # we don't need every column contained in the data for calculating QIs. Following defines which ones to drop
# columns_to_drop = ['First Forename',
#                   'Surname',
#                   'Patient Created Date']


# # assign the column dropping list
# demographic_df = demographic_df.drop(columns=columns_to_drop)

In [55]:
# rename columns appropriately from their key to actual name
column_renaming = {'Chi Number': 'CHI', 
                   'Date Of Birth	': 'Date_Of_Birth',
                   'Health Board Of Residence Description': 'Health_Board_Of_Residence',
                   'Patient Status': 'Patient_Status',
                   'Patient Treatment Centre': 'Treatment_Centre',
                   'Patient Treatment Centre Health Board': 'Treatment_Centre_Health_Board'}


# assign the column renaming list
demographic_df = demographic_df.rename(columns=column_renaming)

In [56]:
# Combine first and second name into a single column
demographic_df.loc[:, 'Name'] = demographic_df['First Forename'].fillna('') + ' ' + demographic_df['Surname'].fillna('')

# drop the initial columns
demographic_df = demographic_df.drop(columns=['First Forename', 'Surname'])

In [57]:
# rename columns appropriately from their key to actual name
column_renaming = {'Chi Number': 'CHI', 
                   'Condition Deleted': 'Condition_Deleted',
                   'Date Condition Diagnosed': 'Date_Condition_Diagnosed',
                   'Condition.Condition': 'Condition',
                   'Sub Condition': 'Sub_Condition',
                   'Condition Outcome Date': 'Condition_Outcome_Date',
                   'Condition Outcome': 'Condition_Outcome'}


 # assign the column renaming list
condition_df = condition_df.rename(columns=column_renaming)                  


In [58]:
# re-add CHIs with missing leading zero
qi_df.loc[:, 'CHI'] = qi_df['CHI'].astype(str).apply(lambda x: x.zfill(10) if len(x) == 9 else x)
demographic_df.loc[:, 'CHI'] = demographic_df['CHI'].astype(str).apply(lambda x: x.zfill(10) if len(x) == 9 else x)
condition_df.loc[:, 'CHI'] = condition_df['CHI'].astype(str).apply(lambda x: x.zfill(10) if len(x) == 9 else x)


In [59]:
# the test patient with a CHI of 1111111111 is contained in this data and must be removed
condition_df = condition_df[condition_df['CHI'] != 1111111111]


# where Condition Deleted is 1, these rows can be removed
condition_df = condition_df[(condition_df['Condition_Deleted'] != 1)]


In [60]:
# filter to only include patients with adrenal insufficiency in the condition columm 
condition_df = condition_df[condition_df['Condition'] == 'ADRENAL INSUFFICIENCY']

In [61]:
# drop duplicate conditions to only include unique patients
condition_df = condition_df.drop_duplicates(subset='CHI')


# reset the index
condition_df = condition_df.reset_index(drop=True)

In [62]:
# we don't need every column contained in the data for calculating QIs. Following defines which ones to drop
columns_to_drop = ['Condition_Deleted']


# assign the column dropping list
condition_df = condition_df.drop(columns=columns_to_drop)

In [63]:
demographic_df['CHI'] = demographic_df['CHI'].astype(condition_df['CHI'].dtype)

In [64]:
ai_demographics_df = pd.merge(demographic_df, condition_df, on='CHI', how='inner')

In [65]:
if qi_df['CHI'].duplicated().any():
    # Duplicates found in the 'CHI' column
    print("Duplicate entries found in the 'CHI' column. Stopping execution.")
    sys.exit()
else:
    # No duplicates found, continue with code
    print("No duplicates found in the 'CHI' column. Continue with the code.")

## if duplicates are found - the following join for merged_df will need to be changed to right i think

No duplicates found in the 'CHI' column. Continue with the code.


### Calculate QIS

#### 1. Initial Emergency Plan

In [66]:
# merge the two dataframes before calculating QIs
merged_df = pd.merge(ai_demographics_df, qi_df, on='CHI', how='left')




In [67]:
# most recent date of record is prioritised, the dataframe is sorted accordingly
merged_df = merged_df.sort_values(by='INITIAL EMERGENCY PLAN', ascending=False)


In [68]:
# assign False if there is no date of initial emergency plan otherwise True
ai_demographics_df['Initial_Emergency_Plan'] = merged_df['INITIAL EMERGENCY PLAN'].notnull()


# reset the index
ai_demographics_df = ai_demographics_df.reset_index(drop=True)

#### 2. Parental Education On IM Injections

In [69]:
# merge the two dataframes before calculating QIs
merged_df = pd.merge(ai_demographics_df, qi_df, on='CHI', how='left')


# most recent date of record if prioritised, the dataframe is sorted accordingly
merged_df = merged_df.sort_values(by='PARENTAL EDUCATION ON IM INJECTIONS', ascending=False)


# drop duplicate rows so we can retrieve a single value for each patient
merged_df = merged_df.drop_duplicates(subset='CHI', keep='first')

In [70]:
# assign False if there is no date of initial emergency plan otherwise True
ai_demographics_df['Parental_Education_On_IM_Injections'] = merged_df['PARENTAL EDUCATION ON IM INJECTIONS'].notnull()


# reset the index
ai_demographics_df = ai_demographics_df.reset_index(drop=True)

#### 3. Relevant Acute Services Notification

In [71]:
# merge the two dataframes before calculating QIs
merged_df = pd.merge(ai_demographics_df, qi_df, on='CHI', how='left')


# most recent date of record if prioritised, the dataframe is sorted accordingly
merged_df = merged_df.sort_values(by='RELEVANT ACUTE SERVICES NOTIFICATION', ascending=False)


# drop duplicate rows so we can retrieve a single value for each patient
merged_df = merged_df.drop_duplicates(subset='CHI', keep='first')

In [72]:
# assign False if there is no date of initial emergency plan otherwise True
ai_demographics_df['Relevant_Acute_Services_Notification'] = merged_df['RELEVANT ACUTE SERVICES NOTIFICATION'].notnull()


# reset the index
ai_demographics_df = ai_demographics_df.reset_index(drop=True)

#### 4. School Education

In [73]:
# merge the two dataframes before calculating QIs
merged_df = pd.merge(ai_demographics_df, qi_df, on='CHI', how='left')


# most recent date of record if prioritised, the dataframe is sorted accordingly
merged_df = merged_df.sort_values(by='SCHOOL EDUCATION', ascending=False)


# drop duplicate rows so we can retrieve a single value for each patient
merged_df = merged_df.drop_duplicates(subset='CHI', keep='first')

In [74]:
# assign False if there is no date of initial emergency plan otherwise True
ai_demographics_df['School_Education'] = merged_df['SCHOOL EDUCATION'].notnull()


# reset the index
ai_demographics_df = ai_demographics_df.reset_index(drop=True)

#### 5. Scottish Ambulance Service Notification

In [75]:
# merge the two dataframes before calculating QIs
merged_df = pd.merge(ai_demographics_df, qi_df, on='CHI', how='left')


# most recent date of record if prioritised, the dataframe is sorted accordingly
merged_df = merged_df.sort_values(by='SCOT AMBULANCE SERVICE NOTIFICATION', ascending=False)


# drop duplicate rows so we can retrieve a single value for each patient
merged_df = merged_df.drop_duplicates(subset='CHI', keep='first')

In [76]:
# assign False if there is no date of initial emergency plan otherwise True
ai_demographics_df['Scottish_Ambulance_Service_Notification'] = merged_df['SCOT AMBULANCE SERVICE NOTIFICATION'].notnull()


# reset the index
ai_demographics_df = ai_demographics_df.reset_index(drop=True)

#### 6. Annual Update of Emergency Plan

In [77]:
# assign todays date to variable
today = pd.to_datetime('today')


# assign date 12 months before today to variable
one_year_ago = today - pd.DateOffset(months=12)


def Annual_Update(row):
    if row['ANNUAL UPDATE OF EMERGENCY PLAN'] >= one_year_ago:
        val = True
    else:
        val = False
    return val

In [78]:
# merge the two dataframes again to begin fresh with calculations
merged_df = pd.merge(ai_demographics_df, qi_df, on='CHI', how='left')


merged_df['ANNUAL UPDATE OF EMERGENCY PLAN'] = pd.to_datetime(merged_df['ANNUAL UPDATE OF EMERGENCY PLAN'], format='%d/%m/%Y')


# group by CHI to get the most recent date
most_recent_update = merged_df.groupby('CHI', as_index=False)['ANNUAL UPDATE OF EMERGENCY PLAN'].max()


# apply function to return true or false based on date updates
most_recent_update['Annual_Emergency_Plan_Update'] = most_recent_update.apply(Annual_Update, axis=1)


In [79]:
# merge the dataframes now that the 6 monthly height recording QI has been calculated
ai_demographics_df = pd.merge(ai_demographics_df, most_recent_update[['CHI', 'Annual_Emergency_Plan_Update']], on='CHI', how='left')



In [80]:
# replace True with 1 and False with 0. Power Bi won't let conditional formatting in matrix visual be done on True/False but it will allow for 1/0
columns_to_update = ['Initial_Emergency_Plan', 'Parental_Education_On_IM_Injections',
                     'Relevant_Acute_Services_Notification', 'School_Education',
                     'Scottish_Ambulance_Service_Notification',
                     'Annual_Emergency_Plan_Update']

ai_demographics_df[columns_to_update] = ai_demographics_df[columns_to_update].fillna(False)


In [81]:
ai_demographics_df[columns_to_update] = ai_demographics_df[columns_to_update].replace({True: 1, False: 0})

In [82]:
# replace True with 1 and False with 0. Power Bi won't let conditional formatting in matrix visual be done on True/False but it will allow for 1/0
columns_to_update = ['Growth_Hormone_Treatment',
                     '6_Monthly_Height_Recording', 'Annual_IGF1_Measurement',
                     'Oestrogen_Replacement_Therapy', 'ORT_Started_Age_10_12',
                     'Has_Core_Data_Set']

ts_demographics_df[columns_to_update] = ts_demographics_df[columns_to_update].fillna(False)


In [83]:
ts_demographics_df[columns_to_update] = ts_demographics_df[columns_to_update].replace({True: 1, False: 0})

### Define dates when data was last refreshed and export to excel

In [85]:
ts_file_path = file_input_path + '/2024-07-30 ENDOCRINE-Report.xlsx'
ai_file_path = file_input_path + '/2024-07-30 SPEG-Dashboard-Data.xlsx'

# Get the last modified time
ts_modified_time = os.path.getmtime(ts_file_path)
ai_modified_time = os.path.getmtime(ai_file_path)

In [86]:
# Convert it to a readable format
ts_modified_time = datetime.fromtimestamp(ts_modified_time)
ai_modified_time = datetime.fromtimestamp(ai_modified_time)

In [87]:
data = {
    'Dataset': ['Turner Syndrome', 'Adrenal Insufficiency'],
    'Date Last Refreshed': [ts_modified_time, ai_modified_time]
}



In [88]:
data_modified_df = pd.DataFrame(data)

In [89]:
data_modified_df.to_excel('cleaned_data/refreshed_dates.xlsx', index=False)

### Export data for Power BI

In [90]:
ai_demographics_df.to_excel('cleaned_data/ai_calculated_qis.xlsx', index=False)
ts_demographics_df.to_excel('cleaned_data/ts_calculated_qis.xlsx', index=False)


### Export Patient data for Health Boards

In [91]:
# following groups together AI and TS QIs into separate grouped dataframes
grouped_ai_health_board_df = ai_demographics_df.groupby('Health_Board_Of_Residence')
grouped_ts_health_board_df = ts_demographics_df.groupby('Health_Board_Of_Residence')



#### Calculate which patients are over 18 with a blank or current patient status


In [92]:
# calculate current date
current_date = pd.to_datetime('today') 


# assign age of each patient to new column
demographic_df['Age'] = np.floor_divide((current_date - demographic_df['Date Of Birth']).dt.days, 365)


# filter patients over 18 with current and blank status
patients_active_over_18 = demographic_df[
    ((demographic_df['Patient_Status'] == 'CURRENT') |
    (pd.isna(demographic_df['Patient_Status']))) &
    (demographic_df['Age'] >= 18)]


# group patients by health board
grouped_patients_active_over_18 = patients_active_over_18.groupby('Health_Board_Of_Residence')

#### Calculate which conditions do not have a date of diagnosis

In [93]:
# somewhere in the process of the endocrine report being produced, the leading zeros of CHI numbers have been ommitted
# the following adds these back in
raw_demographic_df.loc[:, 'Chi Number'] = raw_demographic_df['Chi Number'].astype(str).apply(lambda x: x.zfill(10) if len(x) == 9 else x)
raw_condition_df.loc[:, 'Chi Number'] = raw_condition_df['Chi Number'].astype(str).apply(lambda x: x.zfill(10) if len(x) == 9 else x)


In [94]:
# merge condition and demographic data
all_patients_condition = pd.merge(raw_demographic_df, raw_condition_df, on='Chi Number', how='left') 


# filter conditions with no diagnosis date
condition_no_diagnosis_date = all_patients_condition[pd.isna(all_patients_condition['Date Condition Diagnosed'])]


# group by health board
grouped_patients_no_diagnosis_date = condition_no_diagnosis_date.groupby('Health Board Of Residence Description')

#### Calculate which patients do not have a condition added

In [95]:
# merge condition and demographic data
all_patients_condition = pd.merge(raw_demographic_df, raw_condition_df, on='Chi Number', how='left')


# filter which patients have no condition
patients_no_condition = all_patients_condition[pd.isna(all_patients_condition['Condition.Condition'])]


# group by health board
grouped_patients_no_condition = patients_no_condition.groupby('Health Board Of Residence Description')

In [96]:
import os

for healthboard in grouped_ai_health_board_df.groups.keys():
    try:
        # Create a filename for each health board
        filename = os.path.join(output_directory, f'{healthboard}.xlsx')
        # Create an ExcelWriter object for the current health board
        with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
            # Write AI dataframe to the first sheet
            grouped_ai_health_board_df.get_group(healthboard).to_excel(writer, sheet_name='AI_Patients', index=False)
            
            # Write TS dataframe to the second sheet
            grouped_ts_health_board_df.get_group(healthboard).to_excel(writer, sheet_name='TS_Patients', index=False)

            # Write over 18 to the third sheet
            grouped_patients_active_over_18.get_group(healthboard).to_excel(writer, sheet_name='Patients_18_&_over', index=False)
            
            # Write no diagnosis date dataframe to the fourth sheet
            grouped_patients_no_diagnosis_date.get_group(healthboard).to_excel(writer, sheet_name='Patients_No_Diagnosis_Date', index=False)

            # Write no diagnosis date dataframe to the fourth sheet
            grouped_patients_no_condition.get_group(healthboard).to_excel(writer, sheet_name='Patients_No_Condition', index=False)

    except KeyError:
        print(f"No data found for '{healthboard}' - Skipping to next file")


No data found for 'NHS DUMFRIES & GALLOWAY' - Skipping to next file
